In [ ]:
#hide
from utilities.ipynb_docgen import *
from nbdev import *

# Analyses

> Ongoing analyses


**`light_curves` library functions used here**
- `get_cells`
- `simulate_cells`
- `get_lightcurve`
- `get_bb_partition`
- `partition_cells`
- `fit_cells`
- `flux_plot`


## Introduction

This notebook does not produce a module to add to the library, it a place to run, and report on analyses using the library. It uses local code, shown below.

This is also an exploration of this style of presenting code, and data analysis based on it.

## Geminga analyses

Following Kerr's example, we use the source Geminga to check that this analysis of data gives results consistent with its being constant. As you can see below, the BB analysis finds 16 breaks, which merit further checks. That is followed by a simulation using
Geminga's weight distribution and exposure. The run shown here finds one minor break, a change of 0.1%. 

**Local code**

In [ ]:
#collapse-hide
from light_curves.config import *
from light_curves.bayesian import *
from light_curves.simulation import *
from light_curves.lightcurve import *
from light_curves.cells import *

def bb_overplot(config, lc, bb_fit, ax=None,  **kwargs):
    fig, ax = plt.subplots(1,1, figsize=(12,4)) if not ax else (ax.figure, ax)
    flux_plot(config, lc, ax=ax, colors=(('lightblue', 'sandybrown', 'blue')),**kwargs)
    flux_plot(config, bb_fit, ax=ax, step=True, **kwargs)
    
def simulation(config, source, bb_key=None):
    """Create and analyze a a simulation for the source
    Returns the simulated, and fit light curves
    """

    lc = get_lightcurve(config, source)
    data_cells = get_cells(config, source)

    #  Get the rate from the data
    cq = data_cells.query('e>0.3')
    T, N = np.sum(cq.tw), np.sum(cq.n)
    sflux=lambda t: N/T

    # simulate, then fit cells to create a simulated light curve 
    sim_cells = simulate_cells(config, source, source_flux=sflux  )
    sim_lc  = fit_cells(config, sim_cells) 

    sim_edges = get_bb_partition(config, sim_lc,  key=bb_key) #'simulated_BB_partition_Geminga') 

    # partion, then fit the cells according to the edges
    sim_bb_cells = partition_cells(config, sim_cells, sim_edges);
    sim_bb_fit  = fit_cells(config, sim_bb_cells, )
    return sim_lc, sim_bb_fit

def analyze_data(config, source):
    """
    Analyze data from the source
    
    Returns, data a partitioned light curves
    """
    lc = get_lightcurve(config, source)
    cells = get_cells(config, source)
    edges = get_bb_partition(config, lc, LikelihoodFitness, key='bb-Geminga-test') 
    bb_cells = partition_cells(config, cells, edges);
    bb_lc  = fit_cells(config, bb_cells, )
    return lc, bb_lc

def fit_table(lc, expect=1.0):
    """Generate a summary table from a light curve"""
    fits = lc.fit
    flux = fits.apply(lambda f: f.flux)
    errors = fits.apply(lambda f: (round(f.errors[0]-f.flux,3), round(f.errors[1]-f.flux ,3) ) )
    sigma_dev = fits.apply(lambda f: round(f.poiss.sigma_dev(expect),1) )
    df = lc['t tw n'.split()]; df
    df.loc[:,'flux'] = flux.values.round(4)
    df.loc[:, 'errors'] = errors.values
    df.loc[:, 'sigma_dev'] = sigma_dev.values
    return df

In [ ]:
#collapse-hide
def analysis_plots(name, expect=0.990, simname=''):
    """
    #### {sim} Geminga data
    
    Get, or simulate, the daily binned data, or cells; perform Bayesian Blocks partition;
    make fits to the blocks. <br>(Run at {date})
    
    {output}
    
    This shows the fits to all cells, with the BB fit overlays
    {fig1}
    
    Since this is a constant source, there should be no breaks, that is, only one partition.
    Here is a table of the partition fits:
    
    {df_text}
    The last column represents the compatibility of the flux measurement for each partition
    with the expected value {expect} in equivalent sigma units.
    
    Expand the plot around short, < 100 day partitions.
    {short_check}
    
    {fig2}
    """
    config = Config()
    source = PointSource(name)
    simulated = bool(simname)
    sim= 'Simulated' if simulated else ''
    with capture_print('Analysis output' ) as output:
        if not simulated:
            lc, bb_lc = analyze_data(config, source)
        else: 
            lc, bb_lc = simulation(config, source, bb_key=simname) 

    pd.set_option('display.precision', 3)#, 'display.colheader_justify','left')
    expect
    df = fit_table(bb_lc, expect=expect)
    df_text = monospace(str(df), 'BB fit table', open=True)
        
    plt.rc('font', size=16)
    fig1, ax = plt.subplots(1,1, sharex=True, figsize=(10,4), num=1)
    bb_overplot(config, lc, bb_lc, ax = ax)
    ax.text(0.05, 0.85, name,  transform=ax.transAxes);
    fig1.width=600

    bb_short = bb_lc.query('tw<100'); ns =len(bb_short)
    if ns>0:
        short_check=f'There are {ns} such.'
        rows = (ns+1)//3
        fig2, axx = plt.subplots(3, rows, figsize=(15, 4*rows),sharey=True, sharex=True,
                     gridspec_kw=dict(top=0.85, left=0.08, bottom=0.15, hspace=0.2 ),num=2)
        fig2.width=600
        for t, ax in zip(bb_short.t, axx.flatten()):
            bb_overplot(config, lc, bb_lc, ax=ax, tzero=t, xlim=(-50, +50))
    else:
        fig2=''
        short_check = 'None found.'
    return locals()

if Config().valid:
    nbdoc(analysis_plots, 'Geminga')

/home/burnett/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/burnett/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


####  Geminga data

Get, or simulate, the daily binned data, or cells; perform Bayesian Blocks partition;
make fits to the blocks. <br>(Run at 2020-12-22 13:30)

<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Light curve for Geminga: Saving to cache with key "lightfcurve_Geminga"<br>Cell data for Geminga: Saving to cache with key "cells_Geminga"<br>Photon data: Saving to cache with key "photons_Geminga"<br>Loading  132 months from Arrow dataset /home/burnett/data/dataset<br>....................................................................................................................................<br>	Selected 1,313,726 photons within 5 deg of  (195.13,4.27)<br>	Energies: 100.0-1000000 MeV<br>	Dates:    2008-08-04 15:46 - 2019-08-03 01:17<br>	MJD  :    54682.7          - 58698.1         <br>Load weights from file /mnt/c/users/thbur/OneDrive/fermi/weight_files/Geminga_weights.pkl<br>	Found: PSR J0633+1746 at (195.14, 4.27)<br>	Applyng weights: 240 / 1313726 photon pixels are outside weight region<br>	233109 weights set to NaN<br>binned exposure for source Geminga: Saving to cache with key "binned_exposure_Geminga"<br>exposure for Geminga: Saving to cache with key "exposure_Geminga"<br>Processing 12 S/C history (FT2) files<br>  applying cuts cos(theta) &lt; 0.4,  z &lt; 100<br>	file /home/burnett/work/lat-data/ft2/ft2_2008.fits: 362996 entries, 360944 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2009.fits: 874661 entries, 870446 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2010.fits: 889547 entries, 884697 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2011.fits: 882832 entries, 871672 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2012.fits: 881317 entries, 868109 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2013.fits: 885307 entries, 867342 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2014.fits: 894730 entries, 886570 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2015.fits: 890006 entries, 886086 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2016.fits: 890933 entries, 884823 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2017.fits: 888349 entries, 883761 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2018.fits: 842824 entries, 830723 in GTI<br>	file /home/burnett/work/lat-data/ft2/ft2_2019.fits: 737029 entries, 514657 in GTI<br>	Found 9,609,830 S/C entries:  2,695,715 remain after zenith and theta cuts<br>Calculate exposure using the energy domain 100.0-1000000.0 4 bins/decade<br>2695715 entries, MJD 54683 - 58698<br>Time bins: 4015 intervals of 1 days, in range (54683.0, 58698.0)<br>Loaded 3873 / 4015 cells with exposure &gt; 0.3 for light curve analysis<br>first cell: t                                                    54683.5<br>tw                                                         1<br>e                                                    1.26456<br>n                                                        330<br>w          [244, 214, 174, 5, 246, 150, 187, 65, 91, 215,...<br>S                                                    225.585<br>B                                                    114.707<br>loglike    light_curves.loglike.LogLike:  time 54683.500,...<br>Name: 0, dtype: object<br>Fitting likelihoods with poisson representation<br>Cell data for Geminga: Restoring from cache with key "cells_Geminga"<br>BB edges for...: Saving to cache with key "bb-Geminga-test"<br>Partitioned 3873 cells into 11 blocks, using LikelihoodFitness <br>Loaded 11 / 11 cells with exposure &gt; 0.3 for fitting<br></pre></div> </details>

This shows the fits to all cells, with the BB fit overlays
<div class="nbdoc_image">
<a href="images/analysis_plots_fig_01.png"><figure style="margin-left: 5%">
   <img src="images/analysis_plots_fig_01.png" alt="Figure 1 at images/analysis_plots_fig_01.png" width=600> 
</figure></a></div>


Since this is a constant source, there should be no breaks, that is, only one partition.
Here is a table of the partition fits:

<details open class="nbdoc-description" >  <summary> BB fit table </summary>  <div style="margin-left: 5%"><pre>          t      tw       n   flux           errors  sigma_dev<br>0   55027.5   689.0  208105  0.999  (-0.003, 0.003)        2.9<br>1   55378.5    13.0    3524  1.121  (-0.025, 0.026)        5.3<br>2   55559.5   349.0   95852  0.984  (-0.004, 0.004)       -1.3<br>3   55747.0    26.0    7451  1.117  (-0.017, 0.018)        7.5<br>4   56114.0   708.0  191491  0.991  (-0.003, 0.003)        0.4<br>5   56487.0    38.0   10741  1.078  (-0.014, 0.014)        6.5<br>6   56512.5    13.0    3402  0.901  (-0.021, 0.022)       -4.0<br>7   56858.0   678.0  180858  0.997  (-0.003, 0.003)        2.2<br>8   57197.5     1.0     149  0.535  (-0.066, 0.071)       -5.5<br>9   57913.5  1431.0  367469  0.994  (-0.002, 0.002)        1.8<br>10  58663.0    68.0   11392  1.071  (-0.014, 0.014)        6.1</pre></div> </details>
The last column represents the compatibility of the flux measurement for each partition
with the expected value 0.99 in equivalent sigma units.

Expand the plot around short, < 100 day partitions.
There are 6 such.

<div class="nbdoc_image">
<a href="images/analysis_plots_fig_02.png"><figure style="margin-left: 5%">
   <img src="images/analysis_plots_fig_02.png" alt="Figure 2 at images/analysis_plots_fig_02.png" width=600> 
</figure></a></div>



In [ ]:
#
if Config().valid:
    nbdoc(analysis_plots, 'Geminga', name='analysis_sim',  simname='analysis_plot_sim')

/home/burnett/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/burnett/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


#### Simulated Geminga data

Get, or simulate, the daily binned data, or cells; perform Bayesian Blocks partition;
make fits to the blocks. <br>(Run at 2020-12-22 13:37)

<details  class="nbdoc-description" >  <summary> Analysis output </summary>  <div style="margin-left: 5%"><pre>Light curve for Geminga: Restoring from cache with key "lightfcurve_Geminga"<br>Cell data for Geminga: Restoring from cache with key "cells_Geminga"<br>binned exposure for source Geminga: Restoring from cache with key "binned_exposure_Geminga"<br>Weight histogram for Geminga: Saving to cache with key "weight_hist_Geminga"<br>Photon data: Restoring from cache with key "photons_Geminga"<br><br>	Selected 1,313,726 photons within 5 deg of  (195.13,4.27)<br>	Energies: 100.0-1000000 MeV<br>	Dates:    2008-08-04 15:46 - 2019-08-03 01:17<br>	MJD  :    54682.7          - 58698.1         <br>Load weights from file /mnt/c/users/thbur/OneDrive/fermi/weight_files/Geminga_weights.pkl<br>	Found: PSR J0633+1746 at (195.14, 4.27)<br>	Applyng weights: 240 / 1313726 photon pixels are outside weight region<br>	233109 weights set to NaN<br>Loaded 3873 / 4015 cells with exposure &gt; 0.3 for fitting<br>BB edges for...: Saving to cache with key "analysis_plot_sim"<br>Partitioned 3873 cells into 1 blocks, using LikelihoodFitness <br>Loaded 1 / 1 cells with exposure &gt; 0.3 for fitting<br></pre></div> </details>

This shows the fits to all cells, with the BB fit overlays
<div class="nbdoc_image">
<a href="images/analysis_sim_fig_01.png"><figure style="margin-left: 5%">
   <img src="images/analysis_sim_fig_01.png" alt="Figure 1 at images/analysis_sim_fig_01.png" width=600> 
</figure></a></div>


Since this is a constant source, there should be no breaks, that is, only one partition.
Here is a table of the partition fits:

<details open class="nbdoc-description" >  <summary> BB fit table </summary>  <div style="margin-left: 5%"><pre>         t      tw        n   flux           errors  sigma_dev<br>0  56690.0  4014.0  1114526  0.988  (-0.001, 0.001)       -1.8</pre></div> </details>
The last column represents the compatibility of the flux measurement for each partition
with the expected value 0.99 in equivalent sigma units.

Expand the plot around short, < 100 day partitions.
None found.




## TO DO
- Find out why, for the simulation, and apparently for data, there is a 1% descrepancy between the measured and expected ($\alpha=0$) flux. There are two ways that likelihoods for adjacent cells are combined: the Bayesian Blocks uses tables generated from the Poisson-like representations to each cell, currently with 50 entries, while the fits, to the blocks it recognizes, use the actual weights. I have not carefully compared these.
  - It is apparently the poisson fit when applied to high-statisics data. Take a look at this, perhaps a Gaussian would be easier even.


- Look at the 9 or so intervals detected for the data, of which 5 have measured flux increases around 10%,  more than 4$\sigma$. Possibilities are a problem with the exposure, and a change in the background. The latter can be examined by a 2-D fit with $\beta$ free. Another possibility, fixing $\alpha=0$ and fitting $\beta$ is not (yet) supported.


In [ ]:
!date

Tue Dec 22 13:41:52 PST 2020
